<a href="https://colab.research.google.com/github/HritwikaDuttaMazumdar/lyrics-based-song-recommedation-system-/blob/main/Lyrics%20based%20song%20recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nikhilnayak123/5-million-song-lyrics-dataset")

print("Path to dataset files:", path)

100%|██████████| 3.09G/3.09G [00:40<00:00, 82.8MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/nikhilnayak123/5-million-song-lyrics-dataset/versions/3


In [2]:
import zipfile

# Defining the file path (change 'dataset.zip' to actual file name)
zip_path = "/datalab/archive (1).zip"

# Extracting the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")  # Extract to a new folder

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [3]:
import os
import zipfile

zip_path = "/datalab/archive (1).zip"

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # List the contents of the zip file
    file_list = zip_ref.namelist()

# Print the file list
print(file_list)

['spotify_songs.csv']


In [4]:
import pandas as pd

# Replacing 'filename.csv' with the actual dataset filename
df = pd.read_csv("/content/dataset/spotify_songs.csv")

# Display first few rows
df.head()


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [5]:
# Drop rows with missing lyrics
df = df.dropna(subset=['lyrics'])

# Select only necessary columns
df = df[['track_artist', 'track_name', 'lyrics']]

# Convert lyrics to lowercase
df['lyrics'] = df['lyrics'].str.lower()

print(df.head())


      track_artist                                         track_name  \
0  Barbie's Cradle                                           Pangarap   
1    Steady Rollin                                       I Feel Alive   
2   Bell Biv DeVoe                                             Poison   
3      CeeLo Green  Baby It's Cold Outside (feat. Christina Aguilera)   
4             KARD                                         Dumb Litty   

                                              lyrics  
0  minsan pa nang ako'y napalingon hindi ko alam ...  
1  the trees, are singing in the wind the sky blu...  
2  na yeah, spyderman and freeze in full effect u...  
3  i really can't stay baby it's cold outside i'v...  
4  get up out of my business you don't keep me fr...  


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Transform lyrics into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(df['lyrics'])

print("TF-IDF matrix shape:", tfidf_matrix.shape)


TF-IDF matrix shape: (18194, 5000)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer #Importing TfidfVectorizer to define it in this cell


# Initialize TF-IDF Vectorizer outside of the function.
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000) #Initializing it here makes it available globally in this notebook

def recommend_songs(query, vectorizer, tfidf_matrix, top_n=5): #Added vectorizer as argument
    query = query.lower()  # Convert query to lowercase
    query_vector = vectorizer.transform([query])  # Convert to TF-IDF
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_n:][::-1]

    results = df.iloc[top_indices][['track_artist', 'track_name', 'lyrics']]
    return results

# Get query from user
query = input("Enter lyrics snippet to search for similar songs: ")

#Transform lyrics to TF-IDF outside of the function
tfidf_matrix = vectorizer.fit_transform(df['lyrics']) #this needs to be called before recommend_songs can function, and in the same scope as it.

# Get recommendations, including the vectorizer and tfidf_matrix when calling the function.
recommendations = recommend_songs(query, vectorizer, tfidf_matrix)

# Display results
print("\nTop Recommended Songs:")
print(recommendations)

def recommend_songs(query, top_n=5):
    query = query.lower()  # Convert query to lowercase
    query_vector = vectorizer.transform([query])  # Convert to TF-IDF
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_n:][::-1]

    results = df.iloc[top_indices][['track_artist', 'track_name', 'lyrics']]
    return results

# Get query from user
query = input("Enter lyrics snippet to search for similar songs: ")

# Get recommendations
recommendations = recommend_songs(query)

# Display results
print("\nTop Recommended Songs:")
print(recommendations)


Enter lyrics snippet to search for similar songs: cloud

Top Recommended Songs:
       track_artist                              track_name  \
7206   Dove Cameron  Cloud 9 - Original TV Movie Soundtrack   
2910     Joe Bataan                                My Cloud   
11875         Queen      Some Day One Day - Remastered 2011   
15531   Angie Stone            No More Rain (In This Cloud)   
13552      Yeah Boy     Looking For Sunshine - Original Mix   

                                                  lyrics  
7206   wakin' up, ready for some action strapin' in, ...  
2910   thank you lord for the cloud you made for me i...  
11875  you never heard my song before the music was t...  
15531  no more rain in this cloud no more rain in thi...  
13552  tell me how you feel tell me how you feel tell...  
